In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from scipy.interpolate import CubicSpline
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from gioutils import ezutils as ez
from gioutils.gui.pandas import millifyp as millify
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion, Runner
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-03-02 09:33:56,403 - Authenticating...
2023-03-02 09:33:56,918 - Successfully authenticated


In [5]:
super_clarion(clarion)

In [6]:
runner = Runner(clarion)

### Metrics

In [7]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)


In [8]:
future_months_codes = [
    'F',
    'G',
    'H',
    'J',
    'K',
    'M',
    'N',
    'Q',
    'U',
    'V',
    'X',
    'Z'
]

future_order = [f"{month}{year}" for year in range(0, 12 + 1) for month in future_months_codes]

In [9]:
mid_curve_map = {
    'sfr': [
        # 'sra', 
        # 'srr', 
        # 'srw', 
        '0q', 
        '2q', 
        '3q',
        '4q',
    ],
    'er': [
        '0r', 
        '2r', 
        '3r',
        '4r'
    ]
}

fixing_map = {
    'sfr': 'SOFRRATE Index',
    'er': 'EUR003M Index',
    'rx': 'GTDEM10Y Govt'

}

clarion_ticker_map = {
    'rx': 'BUND EUX',
    'ty': ''
}

clarion_call_put_map = {
    'c': 'Call',
    'p': 'Put'
}

In [ ]:
def get_underlying_ticker(underlying, contract, yk=None):
    yk = yk or 'Comdty'
    return f"{underlying.upper()}{contract.upper() if isinstance(contract, str) else contract} {yk.title()}"

In [ ]:
def get_fut_chain(underlying, contracts=None, yk=None, spot_yield=True):
    contracts = contracts or range(1, 16 + 1)
    yk = yk or 'Comdty'
    securities = [get_underlying_ticker(underlying, contract, yk) for contract in contracts]
    tickers = [*(bq.bdp(securities, {"FUT_CUR_GEN_TICKER": 'ticker'})['ticker'] + ' ' + yk)]
    ticker_map = {contract: ticker for contract, ticker in zip(contracts, tickers)}
    prices = bq.bdp([fixing_map[underlying.lower()], *tickers], {"px_last": 'price', "LAST_TRADEABLE_DT": 'expiry'})
    prices['expiry_code'] = prices['security'].str.replace(underlying.upper(), '').str.split(' ').str[0]
    prices['expiry_code_full'] = prices['expiry_code'].str[0] + str(today().year)[-2] + prices['expiry_code'].str[1]
    prices.iloc[0, -1] = np.nan
    prices.loc[prices['security'] == fixing_map[underlying.lower()], 'expiry'] = today()
    if spot_yield:
        prices.loc[prices['security'] == fixing_map[underlying.lower()], 'price'] = 100 - prices.loc[prices['security'] == fixing_map[underlying.lower()], 'price']
    days = (prices['expiry'] - today()).dt.days
    prices['days'] = days
    return prices

In [ ]:
def get_opt_chain_and_info(underlying):
    chain_ = bq.bds(underlying, 'OPT_CHAIN')
    if chain_.empty:
        return
    chain_ = chain_.squeeze().rename('chain')
    chain = chain_.str.replace(r'\s+', ' ', regex=True).str.split(' ')
    chain = pd.DataFrame.from_dict(dict(zip(chain.index, chain.values))).T.set_axis(['code', 'strike', 'yk'], axis=1)
    chain['call_put'] = chain['code'].str[-1]
    chain['code'] = chain['code'].str[:-1]
    chain['strike'] = chain['strike'].astype(float)
    chain['orig'] = chain_
    
    opt_chain = chain.drop_duplicates('code').set_index('code')['orig']
    opt_expiry = bq.bdp(opt_chain, {'LAST_TRADEABLE_DT': 'expiry', 'CRNCY': 'ccy'}).rename({'security': 'orig'}, axis=1)
    
    chain['ccy'] = opt_expiry['ccy'].drop_duplicates().squeeze()
    
    def get_strike_increment(df):
        return df['strike'].head(2).diff().dropna().squeeze()
    increments = chain.groupby('code', sort=False).apply(get_strike_increment).rename('inc')
    opt_info = opt_expiry.merge(opt_chain.reset_index(), on='orig').merge(increments, on='code')
    opt_info['expiry_code'] = opt_info['code'].str[-2:]
    opt_info['expiry_code_full'] = opt_info['expiry_code'].str[0] + str(today().year)[-2] + opt_info['expiry_code'].str[1]
    
    return chain, opt_info

In [ ]:
str(today().year)[-2]

In [ ]:
underlying = 'rx'
opt_type = 'P'
contracts = [2]
underlying_ticker = get_underlying_ticker('rx', 'm3')

In [ ]:
fut_prices = get_fut_chain(underlying, contracts, spot_yield=False)

In [ ]:
opt_chain, opt_info = get_opt_chain_and_info(underlying_ticker)

In [ ]:
opt_chain.query(f'code == "{opt["code"]}" and call_put == "{opt_type}"')

In [ ]:
instruments = [] 

for i, fut in fut_prices.iloc[1:, :].iterrows():
    for j, opt in opt_info.iterrows():
        options = opt_chain.query(f'code == "{opt["code"]}" and call_put == "{opt_type}"')
        for k, opt_strike in options.iterrows():
            instrument = i = clarion.instruments.RatesBondFutureOption(
                currency=opt['ccy'],
                instrument_name=clarion_ticker_map.get(underlying.lower()).upper(),
                expiry=opt['expiry_code_full'].upper(),
                tenor=fut['expiry_code_full'].upper(),
                strike=opt_strike['strike'],
                option_type=clarion_call_put_map[opt_strike['call_put'].lower()],
                notional=100,
            )
            instruments.append(instrument)

In [ ]:
data = runner.run(
    metrics=[*clarion_metrics.values()], 
    data_list=instruments
)

In [ ]:
bp_shift = 10 

In [ ]:
data['pv_scenario_chg'] = data['delta'] * bp_shift + 0.5 * data['gamma'] * (bp_shift ** 2)

In [ ]:
data['pv_scenario_pct'] = data['pv_scenario_chg'] / data['pv']

In [ ]:
cols = [
    'description',
    'pv',
    'pv_scenario_chg',
    'pv_scenario_pct',
    'delta',
    'gamma',
    
]

In [ ]:
data.loc[: , cols].sort_values('pv_scenario_pct', ascending=False).to_clipboard()

In [ ]:
data['pv']

In [ ]:
fut_prices

In [ ]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

In [ ]:
usd_curve_shift = clarion.create_shift('RatesCurveShift', 'relative', 0.001, {'curveFamily': 'USD'})
eur_vol_shift = clarion.create_shift('RatesVolShift', 'relative', 0.001, {'curveFamily': 'EUR'})
eur_curve_shift = clarion.create_shift('RatesCurveShift', 'relative', 0.001, {'curveFamily': 'EUR'})
# scenario_rates = clarion.create_scenario('ScenRates', [usd_curve_shift, eur_curve_shift])
scenario_rates = clarion.create_scenario('ScenRates', [eur_curve_shift, eur_vol_shift])

In [ ]:
eur_vol_shift = clarion.create_shift('RatesVolShift', 'relative', 0.001, {'curveFamily': 'EUR'})

In [ ]:
eur_vol_shift

In [ ]:
response['DFRAME']

In [ ]:
i = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='1y',
            tenor='10y',
            strike='a',
            swaption_type='Pay',
            notional=100e6,
            settle_type='Physical Cleared LCH',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ).trade

In [ ]:
#Run the fx scenario on the fx positions
response = clarion.risks.run(
    metrics=[clarion_metrics[metric] for metric in ['delta', 'pv', 'local_pv']], 
    data_list=clarion.create_scenario_population('rates', trade_list=[i]),
    scenarios=[scenario_rates], 
    # asof_date='2022-09-01', 
    market_mode='live'
)
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
clarion.create_scenario_population('rates', trade_list[i.trade for in in instruments][:5])

In [ ]:
#Run the fx scenario on the fx positions
response = clarion.risks.run(
    metrics=[clarion_metrics[metric] for metric in ['delta', 'pv', 'local_pv']], 
    data_list=[i][:5],
    scenarios=[scenario_rates], 
    # asof_date='2022-09-01', 
    market_mode='live'
)
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

In [ ]:
instruments[0].trade.instrument.instrument_data

In [ ]:
i = clarion.create_trade(
    id='a',
    trade_type='RatesBondFutureOption',
    instrument_data=instruments[0].trade.instrument.instrument_data,
    asset_class='Rates'
)

In [19]:
from itertools import product, chain

In [39]:
days = pd.date_range(today() + BDay(7), periods=30, freq='B')
strikes1 = [f"a + {i}"  for i in range(5, 80 + 5, 5)]
strikes2 = [f"a + {i + 10}"  for i in range(5, 80 + 5, 5)]
vols = np.arange(60, 160 + 10, 10) / (252 ** 0.5)

In [40]:
df1 = pd.DataFrame(product(days, strikes1, vols), columns=['expiry', 'strike', 'vol'])
df1['group'] = range(len(df1))

In [41]:
df2 = pd.DataFrame(product(days, strikes2, vols), columns=['expiry', 'strike', 'vol'])
df2['group'] = range(len(df2))

In [42]:
df1 = df1.assign(
    ccy='USD',
    type='oswp',
    gen='USD SOFR',
    tenor='7y',
    pay_rec='pay',
    notional=100,
    coll='USD_bilat',
    cash_set='pc_lch',
    pay_set='s',
    prem=np.nan,
)

df2 = df2.assign(
    ccy='USD',
    type='oswp',
    gen='USD SOFR',
    tenor='7y',
    pay_rec='pay',
    notional=-100,
    coll='USD_bilat',
    cash_set='pc_lch',
    pay_set='s',
    prem=np.nan,
)

In [47]:
df = pd.concat([df1, df2]).sort_values(by=['group', 'notional'], ascending=[True, False])

In [48]:
df = df.loc[
    :, [
        'ccy',
        'type',
        'gen',
        'expiry',
        'tenor',
        'strike',
        'pay_rec',
        'notional',
        'coll',
        'cash_set',
        'pay_set',
        'prem',
        'vol',
        'group',
    ]
]

In [49]:
df

ccy  type       gen     expiry tenor  strike pay_rec  notional  \
0     USD  oswp  USD SOFR 2023-03-13    7y   a + 5     pay       100   
0     USD  oswp  USD SOFR 2023-03-13    7y  a + 15     pay      -100   
1     USD  oswp  USD SOFR 2023-03-13    7y   a + 5     pay       100   
1     USD  oswp  USD SOFR 2023-03-13    7y  a + 15     pay      -100   
2     USD  oswp  USD SOFR 2023-03-13    7y   a + 5     pay       100   
...   ...   ...       ...        ...   ...     ...     ...       ...   
5277  USD  oswp  USD SOFR 2023-04-21    7y  a + 90     pay      -100   
5278  USD  oswp  USD SOFR 2023-04-21    7y  a + 80     pay       100   
5278  USD  oswp  USD SOFR 2023-04-21    7y  a + 90     pay      -100   
5279  USD  oswp  USD SOFR 2023-04-21    7y  a + 80     pay       100   
5279  USD  oswp  USD SOFR 2023-04-21    7y  a + 90     pay      -100   

           coll cash_set pay_set  prem     vol  group  
0     USD_bilat   pc_lch       s   NaN  3.7796      0  
0     USD_bilat   pc_lch       s   NaN  3.7796      0  
1     USD_bilat   pc_lch       s   NaN  4.4096      1  
1     USD_bilat   pc_lch       s   NaN  4.4096      1  
2     USD_bilat   pc_lch       s   NaN  5.0395      2  
...         ...      ...     ...   ...     ...    ...  
5277  USD_bilat   pc_lch       s   NaN  8.8192   5277  
5278  USD_bilat   pc_lch       s   NaN  9.4491   5278  
5278  USD_bilat   pc_lch       s   NaN  9.4491   5278  
5279  USD_bilat   pc_lch       s   NaN 10.0791   5279  
5279  USD_bilat   pc_lch       s   NaN 10.0791   5279  

[10560 rows x 14 columns]

In [51]:
df.drop('group', axis=1).to_clipboard(index=False, header=False)

In [51]:
df[:5].to_clipboard(index=False, header=False)

In [130]:
data = pd.read_clipboard()

In [52]:
x1 = pd.read_clipboard()
x1['days'] = 1

In [55]:
x2 = pd.read_clipboard()
x2['days'] = 2

In [56]:
x3 = pd.read_clipboard()
x3['days'] = 3

In [57]:
x4 = pd.read_clipboard()
x4['days'] = 4

In [58]:
x5 = pd.read_clipboard()
x5['days'] = 5

In [139]:
data = pd.concat([x1, x1, x3, x4, x5])

In [140]:
data['group'] = sorted([*range(len(data) // 2)] * 2)

In [141]:
cols = [
    'days',
    'Generator',
    'Expiry/Start', 
    'Tenor', 
    ' Strike',
    'Strike',
    ' Pay/Rec', 
    'Notional(m)',
    'Prem(m)', 
    'VolOver',
    'Fwd', 
    'Spot',
    'Vol', 
    'RealVol',
    'PV$',
    'Perf',
    'PnL$',
    'group',
]

In [142]:
data['Expiry/Start'] = pd.to_datetime(data['Expiry/Start'])
data['Perf'] = data['Perf'].str.replace('%', '').astype(float)
data['PV$'] = data['PV$'].str.replace(',', '').astype(float)
data['PnL$'] = data['PnL$'].str.replace(',', '').astype(float)

In [143]:
data_ = data.loc[:, cols]

In [170]:
groups = []
for index, group in data_.groupby('group', sort=False):
    g = group.iloc[0, :]
    # g[' Strike'] = g[' Strike'] + ' / ' + group.iloc[1, :][' Strike']
    # g['Strike'] = str(g['Strike']) + ' / ' + str(group.iloc[1, :]['Strike'])
    # g['PV$'] = group['PV$'].sum()
    # g['PnL$'] = group['PnL$'].sum()
    # g['Perf'] = g['PnL$'] / g['PV$'] * 100
    groups.append(g.to_frame().T)

In [171]:
groups = pd.concat(groups)

In [172]:
groups_ = groups.query('`PV$` > 1000')

In [174]:
mean = groups_.groupby(
    [
        'Expiry/Start',  
        ' Strike', 
        'VolOver',
    ],
    as_index=False,
)['Perf'].mean()

In [175]:
mean = mean.sort_values('Perf', ascending=False).dropna()

In [176]:
best_mean = mean.groupby(['Expiry/Start', 'VolOver'], as_index=False, sort=False).first()

In [177]:
best_mean['VolOver'] = (best_mean['VolOver'] * (252 ** 0.5)).astype(int)
best_mean['Perf'] = best_mean['Perf'].round(0)

In [178]:
best_mean.to_clipboard(index=False)

In [122]:
best_mean.pivot(index='VolOver', columns='Expiry/Start', values='Perf')

Expiry/Start  2023-03-10  2023-03-13  2023-03-14  2023-03-15  2023-03-16  \
VolOver                                                                    
60                   286         272         256         243         231   
70                   181         172         164         157         150   
80                   129         123         117         112         109   
90                    98          94          90          86          83   
100                   79          75          72          69          67   
110                   65          62          59          57          55   
120                   55          52          50          48          47   
130                   46          45          43          42          40   
140                   39          39          38          36          35   
150                   34          33          33          32          31   
160                   29          29          29          28          28   

Expiry/Start  2023-03-17  2023-03-20  2023-03-21  2023-03-22  2023-03-23  ...  \
VolOver                                                                   ...   
60                   219         208         199         191         182  ...   
70                   145         139         133         129         124  ...   
80                   104         101          98          94          91  ...   
90                    80          78          75          73          71  ...   
100                   64          63          61          59          58  ...   
110                   53          52          50          49          48  ...   
120                   45          44          43          42          41  ...   
130                   39          38          37          36          35  ...   
140                   34          33          32          32          31  ...   
150                   30          29          29          28          27  ...   
160                   27          26          26          25          25  ...   

Expiry/Start  2023-04-07  2023-04-10  2023-04-11  2023-04-12  2023-04-13  \
VolOver                                                                    
60                   130         130         127         124         120   
70                    93          93          91          89          87   
80                    71          71          70          68          67   
90                    56          56          55          54          53   
100                   46          46          46          45          44   
110                   39          39          38          38          37   
120                   33          33          33          32          32   
130                   29          29          29          28          28   
140                   26          26          25          25          25   
150                   23          23          23          22          22   
160                   21          21          20          20          20   

Expiry/Start  2023-04-14  2023-04-17  2023-04-18  2023-04-19  2023-04-20  
VolOver                                                                   
60                   117         115         112         109         107  
70                    85          83          82          80          78  
80                    65          64          63          62          61  
90                    52          52          51          50          49  
100                   43          43          42          41          41  
110                   37          36          35          35          34  
120                   31          31          30          30          30  
130                   27          27          27          26          26  
140                   24          24          24          23          23  
150                   22          21          21          21          21  
160                   19          19          19          19        

In [81]:
data['Perf'].str.replace('%', '').astype(float)

0       247.3000
1       187.2000
2       149.1000
3            123
4       104.1000
          ...   
26395          4
26396     3.2000
26397     2.4000
26398     1.7000
26399     1.1000
Name: Perf, Length: 26400, dtype: float64

In [169]:
8.32 * 252 ** 0.5

132.07590544834437